# 1. 시즌 챔피언 정보

앞서 배웠던 웹 스크래핑 내용을 토대로 특정 유저의 특정 시즌 랭크 플레이 챔피언 정보를 스크래핑 해보았다.

In [1]:
import time
from tqdm import tqdm
import pandas as pd
from urllib.request import Request, urlopen
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

In [2]:
# headless selenium
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')

In [3]:
request_header = {
    
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.1 Safari/605.1.15",
    "Accept-Language": "ko-KR,ko;q=0.9",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8"
}

In [4]:
def refresh(userName):
    userName2 = "+".join(userName.split())
    # 유저별 검색 결과 url
    url = f"https://www.op.gg/summoner/champions/userName={userName2}"
    driver = webdriver.Chrome("/Users/augustin/OneDrive - 서울과학기술대학교/21년 2학기/기계학습/프로젝트/chromedriver", options=options)
    driver.get(url)
    time.sleep(1)

    driver.find_element_by_xpath('//*[@id="SummonerRefreshButton"]').click()
    driver.quit()
    return

In [5]:
def my_champion(userName):
    # 빈 데이터 프레임
    data = pd.DataFrame()
    
    # 띄어쓰기 있는 유저이면 +로 구분
    userName2 = "+".join(userName.split())

    # 유저별 검색 결과 url
    url = f"https://www.op.gg/summoner/champions/userName={userName2}"
    
    response = requests.get(url, headers=request_header)
    

    # url 정보 저장
    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html')


    else : 
        print(response.status_code)

    # 플레이한 챔피언 데이터 테이블
    champ_info = soup.find("div", attrs= {"class":"Content tabItems"})
    champ_info2 = champ_info.find("div", attrs= {"class":"tabItem season-18"})
    champ_info3 = champ_info2.find("tbody", attrs= {"class":"Body"})
    champ_info4 = champ_info3.find_all("tr")
    
    # 각 태그 지정해서 원하는 정보 추출
    for i in range(len(champ_info4)):
        lst = []
        # 챔피언 이름
        lst.append(champ_info4[i].find("td", attrs= {"class":"ChampionName Cell"}).get_text().strip())

        # 승/패 (단, 한번도 이기거나 진적이 없으면 해당 class가 없음)
        if champ_info4[i].find("div", attrs= {"class":"Text Left"}) == None:
            lst.append(0)
        else:
            lst.append(champ_info4[i].find("div", attrs= {"class":"Text Left"}).get_text().strip()[:-1])

        if champ_info4[i].find("div", attrs= {"class":"Text Right"}) == None:
            lst.append(0)
        else:
            lst.append(champ_info4[i].find("div", attrs= {"class":"Text Right"}).get_text().strip()[:-1])

        # K/D/A
        KDA = champ_info4[i].find("div", attrs= {"class":"KDA"}).get_text().strip().split()
        lst.append(KDA[0])
        lst.append(KDA[2])
        lst.append(KDA[4])
        
        # 기타 정보
        else_info = champ_info4[i].find_all("td", attrs={"class":"Value Cell"})
        for i in else_info:
            info = i.get_text().strip().split()
            if len(info) == 0:
                lst.append(0)
            else:
                lst.append(info[0])
        
        # 데이터 프레임
        champ_df = pd.DataFrame(lst)
        data = pd.concat([data, champ_df], axis=1)
    
    # 데이터 프레임 형태, 타입 설정
    data = data.T
    data.index = range(data.shape[0])
    data.columns = [
        "champion", "win", "lose", "kill", "death", "assist", 
        "gold", "cs", "max_kill", "max_death", "avg_damage", "avg_damaged", "double", "triple", "quadra", "penta"
    ]
    
    data["win"] = data["win"].astype(int)
    data["lose"] = data["lose"].astype(int)
    data["kill"] = data["kill"].astype(float)
    data["death"] = data["death"].astype(float)
    data["death"] = data.death.replace(0,1)
    data["assist"] = data["assist"].astype(float)
    data["gold"] = data["gold"].apply(lambda x: int(x.replace(",","")))
    data["cs"] = data["cs"].astype(float)
    data["max_kill"] = data["max_kill"].astype(int)
    data["max_death"] = data["max_death"].astype(int)
    data["avg_damage"] = data["avg_damage"].apply(lambda x: int(x.replace(",","")))
    data["avg_damaged"] = data["avg_damaged"].apply(lambda x: int(x.replace(",","")))
    data["double"] = data["double"].astype(int)
    data["triple"] = data["triple"].astype(int)
    data["quadra"] = data["quadra"].astype(int)
    data["penta"] = data["penta"].astype(int)
    



    return data

In [6]:
matchData = pd.read_csv('/Users/augustin/OneDrive - 서울과학기술대학교/21년 2학기/기계학습/프로젝트/bsg_final_korean_df.csv')
matchData.drop('Unnamed: 0', inplace=True, axis=1)

In [6]:
#맥용
import pickle

gold = pd.read_csv("/Users/augustin/OneDrive - 서울과학기술대학교/21년 2학기/기계학습/프로젝트/gold.csv", encoding = "cp949")
gold.drop('Unnamed: 0', inplace=True, axis=1)
plat = pd.read_csv("/Users/augustin/OneDrive - 서울과학기술대학교/21년 2학기/기계학습/프로젝트/plat.csv", encoding = "cp949")
plat.drop('Unnamed: 0', inplace=True, axis=1)

name_gold = []

for i in range(1,3):
    for j in range(1,6):
        name_gold.append(gold["t"+str(i)+"_p"+str(j)+"_summoner_name"].tolist())


name_plat = []

for i in range(1,3):
    for j in range(1,6):
        name_plat.append(plat["t"+str(i)+"_p"+str(j)+"_summoner_name"].tolist())


name_gold = sum(name_gold, [])
name_plat = sum(name_plat, [])

name_gold = list(set(name_gold))
name_plat = list(set(name_plat))

In [7]:
def add_summoner_info(userName):
    result = my_champion(userName)
    final_result = pd.DataFrame()
    final_result['champion'] = result['champion']
    final_result['kda'] = (result['kill'] + result['assist']) / result['death']
    final_result['winRate'] = (result['win'] / (result['win'] + result['lose'])) * 100
    final_result['fiveGame_played'] = 0
    for q in range(0, len(result['win'])):
        if((result['win'][q] + result['lose'][q]) >= 5):
            final_result['fiveGame_played'][q] = 1
    return final_result


In [8]:
a = name_gold[:3000]
b = name_gold[3000:6000]
c = name_gold[6000:9000]
d = name_gold[9000:12000]
e = name_gold[12000:15000]
f = name_gold[15000:]

In [12]:
name_list2 = a

for i in tqdm(range(0, len(name_list2))):
    try:
        refresh(name_list2[i])

    except:
        continue

 26%|██▌       | 770/3000 [6:12:40<1:54:04,  3.07s/it]

In [33]:
name_list = f
temp_data_list = []

for i in tqdm(range(0, len(name_list))):
    try:
        temp = add_summoner_info(name_list[i])
        temp['summoner_name'] = name_list[i]
        temp_data_list.append(temp)
    except:
        continue

result = pd.concat(temp_data_list)
result.reset_index(inplace=True)
result.drop('index', inplace=True, axis=1)

result

  0%|          | 2/2123 [00:02<43:09,  1.22s/it]/var/folders/rw/4fycx7992w15h0xsbl23zqwr0000gn/T/ipykernel_7096/1475967671.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_result['fiveGame_played'][q] = 1
100%|██████████| 2123/2123 [39:03<00:00,  1.10s/it]


,champion,kda,winRate,fiveGame_played,summoner_name
0,이즈리얼,1.873563,66.666667,0,머호야
1,베인,1.686567,33.333333,0,머호야
2,노틸러스,5.500000,100.000000,0,머호야
3,케이틀린,4.666667,100.000000,0,머호야
4,미스 포츈,2.722222,100.000000,0,머호야
...,...,...,...,...,...
24534,다리우스,1.444444,0.000000,0,뚝딱 오 른
24535,탈론,1.000000,0.000000,0,뚝딱 오 른
24536,아칼리,0.500000,0.000000,0,뚝딱 오 른
24537,신드라,0.125000,0.000000,0,뚝딱 오 른


In [34]:
gold_6 = result


In [35]:
gold_6

,champion,kda,winRate,fiveGame_played,summoner_name
0,이즈리얼,1.873563,66.666667,0,머호야
1,베인,1.686567,33.333333,0,머호야
2,노틸러스,5.500000,100.000000,0,머호야
3,케이틀린,4.666667,100.000000,0,머호야
4,미스 포츈,2.722222,100.000000,0,머호야
...,...,...,...,...,...
24534,다리우스,1.444444,0.000000,0,뚝딱 오 른
24535,탈론,1.000000,0.000000,0,뚝딱 오 른
24536,아칼리,0.500000,0.000000,0,뚝딱 오 른
24537,신드라,0.125000,0.000000,0,뚝딱 오 른


In [36]:
gold_6.to_csv('gold_6.csv')

In [37]:
gold_1

,champion,kda,winRate,fiveGame_played,summoner_name
0,시비르,2.736842,50.000000,1,유노윤모
1,빅토르,2.151515,42.857143,1,유노윤모
2,볼리베어,1.494845,33.333333,1,유노윤모
3,마오카이,1.066667,0.000000,0,유노윤모
4,모르가나,2.416667,100.000000,0,유노윤모
...,...,...,...,...,...
35050,카타리나,1.500000,0.000000,0,돈슨은넥슨
35051,신드라,3.000000,0.000000,0,돈슨은넥슨
35052,올라프,1.125000,0.000000,0,돈슨은넥슨
35053,다리우스,0.312500,0.000000,0,돈슨은넥슨


In [39]:
gold_final = pd.concat([gold_1, gold_2, gold_3, gold_4, gold_5, gold_6])

In [42]:
gold_final

,champion,kda,winRate,fiveGame_played,summoner_name
0,시비르,2.736842,50.000000,1,유노윤모
1,빅토르,2.151515,42.857143,1,유노윤모
2,볼리베어,1.494845,33.333333,1,유노윤모
3,마오카이,1.066667,0.000000,0,유노윤모
4,모르가나,2.416667,100.000000,0,유노윤모
...,...,...,...,...,...
24534,다리우스,1.444444,0.000000,0,뚝딱 오 른
24535,탈론,1.000000,0.000000,0,뚝딱 오 른
24536,아칼리,0.500000,0.000000,0,뚝딱 오 른
24537,신드라,0.125000,0.000000,0,뚝딱 오 른


In [44]:
gold_final.reset_index(inplace=True)
gold_final.drop('index', inplace=True, axis=1)
gold_final

,champion,kda,winRate,fiveGame_played,summoner_name
0,시비르,2.736842,50.000000,1,유노윤모
1,빅토르,2.151515,42.857143,1,유노윤모
2,볼리베어,1.494845,33.333333,1,유노윤모
3,마오카이,1.066667,0.000000,0,유노윤모
4,모르가나,2.416667,100.000000,0,유노윤모
...,...,...,...,...,...
204151,다리우스,1.444444,0.000000,0,뚝딱 오 른
204152,탈론,1.000000,0.000000,0,뚝딱 오 른
204153,아칼리,0.500000,0.000000,0,뚝딱 오 른
204154,신드라,0.125000,0.000000,0,뚝딱 오 른


In [45]:
gold_final.to_csv('gold_final.csv')